# Skin lesion segmentation #
This project has been made as a final assesment of the course Advanced Image Analysis in the University of Cassino and Southern Lazio by Elizaveta Genke, Mladen Rakic, Liliana Valencia Rodriguez, Daria Zotova.

In [ ]:
import cv2
import numpy as np
import glob
import os

from matplotlib import pyplot as plt

from skimage import io 
from skimage import img_as_ubyte
from skimage import color
from skimage.color import rgb2gray
from skimage.measure import label
from skimage.measure import regionprops
from skimage import morphology
from skimage.morphology import disk
from skimage.morphology import rectangle
from skimage import filters
from skimage.filters import gaussian
from skimage.transform import rescale

import scipy
import scipy.ndimage

In [ ]:
def hair_removal (img_input):
    
    # input arguments:
    # img_input - original color image
    #
    # output arguments:
    # img_output - color image with hairs suppressed 
    
    img_gray = color.rgb2gray(img_input)
    
    # blur the original image
    img_gray_blurred = cv2.GaussianBlur(img_gray,(5,5),0)

    # compute laplacian to obtain a sharpened result
    img_laplacian = scipy.ndimage.filters.laplace(img_gray_blurred)
    img_subtraction = img_gray_blurred - img_laplacian
    
    # perform bottom hat transformation by rotating line kernel 
    kernel_line = np.ones((10,50),np.uint8)
    img_bottom_hat_1 = cv2.morphologyEx(img_subtraction, cv2.MORPH_BLACKHAT, kernel_line)
    kernel_line_rotated_45 = scipy.ndimage.interpolation.rotate(kernel_line, 45)
    img_bottom_hat_2 = cv2.morphologyEx(img_subtraction, cv2.MORPH_BLACKHAT, kernel_line_rotated_45)
    kernel_line_rotated_90 = scipy.ndimage.interpolation.rotate(kernel_line_rotated_45, 45)
    img_bottom_hat_3 = cv2.morphologyEx(img_subtraction, cv2.MORPH_BLACKHAT, kernel_line_rotated_90)
    img_addition = img_bottom_hat_1 + img_bottom_hat_2 + img_bottom_hat_3
    
    # threshold the obtained result
    threshold = np.amax(img_addition) * 0.2
    ret, img_binary = cv2.threshold(img_addition,threshold,1.0,cv2.THRESH_BINARY)
    
    # dilate the binary image
    kernel_dilation = np.ones((1,10),np.uint8)
    img_dilated = cv2.dilate(img_binary,kernel_dilation,iterations = 1)
    img_dilated = np.uint8(img_dilated)
    
    # perform the inpainting technique
    img_output = cv2.inpaint(img_input, img_dilated, 20, cv2.INPAINT_TELEA)
    
    return img_output

In [ ]:
def meanshift_segmentation (img_input):
    
    # input arguments:
    # img_input - color image with hairs suppressed 
    #
    # output arguments:
    # img_output - binary meanshift segmented image
    
    # perform meanshift filtering
    img_meanshift = cv2.pyrMeanShiftFiltering(img_input, 30, 40)
    img_gray = color.rgb2gray(img_meanshift)
    
    # convert float grayscale image into int
    img_min = img_gray.min()
    img_max = img_gray.max()
    img_copy = img_gray.copy()

    for i in range (img_gray.shape[0]):
        for j in range (img_gray.shape[1]):
            img_copy[i][j] = (img_gray[i][j] - img_min) / (img_max - img_min) * 255
    img_copy = np.uint8(img_copy)
    
    # remove dark regions based on histogram
    hist_values, bins_edges = np.histogram(img_copy,256)
    if hist_values[0] > 10000:
        img_copy = dark_region_removal(img_copy)
    
    ret,img_binary = cv2.threshold(img_copy,120,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_output = 255 - img_binary
    
    return img_output

In [ ]:
def watershed_segmentation (img_original,img_hairless):
    
    # input arguments:
    # img_original - original color image
    # img_hairless - color image with hairs suppressed
    #
    # output arguments:
    # img_output - binary watershed segmented image

    # convert to binary and change the data type
    img_hairless_gray = color.rgb2gray(img_hairless)
    
    img_min = img_hairless_gray.min()
    img_max = img_hairless_gray.max()
    img_copy = img_hairless_gray.copy()
    
    for i in range (img_hairless_gray.shape[0]):
        for j in range (img_hairless_gray.shape[1]):
            img_copy[i][j] = (img_hairless_gray[i][j] - img_min) / (img_max - img_min) * 255
    img_copy = np.uint8(img_copy)
    
    # remove dark regions based on histogram
    hist_values, bins_edges = np.histogram(img_copy,256)
    if hist_values[0] > 10000:
        img_copy = dark_region_removal(img_copy)

    ret,img_binary = cv2.threshold(img_copy,120,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_binary = 255 - img_binary

    # find foreground area
    dist_transform = cv2.distanceTransform(img_binary,cv2.DIST_L2,0)
    ret, foreground = cv2.threshold(dist_transform,0.05*dist_transform.max(),255,0)

    # find background region
    foreground = np.uint8(foreground)
    background = cv2.subtract(img_binary,foreground)

    # label markers
    ret, markers = cv2.connectedComponents(foreground)

    # add one to all labels so that background is not 0, but 1
    markers = markers+1
    
    # mark the region of background with zero
    markers[background == 255] = 0
    
    # apply watershed 
    markers = cv2.watershed(img_original,markers)
    img_original[markers == -1] = [255,0,0]
    
    # change the data type of the output from int32 to uint8
    min_marker = markers.min()
    max_marker = markers.max()
    new_marker = markers.copy()

    for i in range (markers.shape[0]):
        for j in range (markers.shape[1]):
            new_marker[i][j] = (markers[i][j]-min_marker)/(max_marker-min_marker)*255
        
    new_marker = np.uint8(new_marker)
    ret,img_binary_final = cv2.threshold(img_binary,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # perform morphological opening
    kernel = np.ones((10,10),np.float32)/100
    img_output = cv2.morphologyEx(img_binary_final,cv2.MORPH_OPEN, kernel)
    
    return img_output

In [ ]:
def dark_region_removal (img_input):
    
    # input arguments:
    # img_input - grayscale image
    #
    # output arguments:
    # img_output - grayscale image with dark regions suppressed
    
    for i in range (img_input.shape[0]):
        for j in range (img_input.shape[1]):
            if (img_input[i][j] < 50):
                img_input[i][j] = 125
    
    img_output = img_input
    
    return img_output

In [ ]:
def region_extraction (img_input):
    
    # input arguments:
    # img_input - binary image 
    #
    # output arguments:
    # img_output - segmented binary image
    
    # remove small holes and objects in input image
    img_no_holes = morphology.remove_small_holes(img_input)
    img_no_objects = morphology.remove_small_objects(img_no_holes)
   
    # extract region properties
    region_labels, labels_idx = label(img_no_objects, connectivity = 2, return_num = True)
    props = regionprops(region_labels)

    # define centroids for all regions and the central pixel as a target for selection
    centroids = [prop.centroid for prop in props]
    number_rows = img_input.shape[0]
    central_row = np.round(number_rows / 2)
    number_cols = img_input.shape[1]
    central_col = np.round(number_cols / 2)
    
    img_max_area = np.zeros((img_no_objects.shape[0],img_no_objects.shape[1]),np.uint8)
    center = (central_row,central_col)
    index = 0
    min_dist = 10000 
   
    # select the region closest to the central pixel by finding the nearest white pixel
    for k in range(len(centroids)):     
        coords = props[k].coords
        nearest_coords = nearest_region_selection(coords,center)
        current_dist = ((central_row - nearest_coords[0]) ** 2 + (central_col - nearest_coords[1]) ** 2) ** 0.5
        if current_dist < min_dist:
            min_dist = current_dist
            index = k
            
    # find the bounding box of the selected region        
    bounding_box = props[index].bbox
    img_max_area[bounding_box[0]:bounding_box[2],bounding_box[1]:bounding_box[3]] = props[index].image

    # correct the result by filling holes in the output
    img_floodfill = img_max_area.copy()
    height, width = img_input.shape[:2]
    mask = np.zeros((height + 2,width + 2),np.uint8)
    cv2.floodFill(img_floodfill,mask,(0,0),255)
    img_floodfill_inv = cv2.bitwise_not(img_floodfill)
    img_output = img_max_area | img_floodfill_inv

    return img_output

In [ ]:
def nearest_region_selection (coords,target):
    
    # input arguments:
    # coords - coordinates of the region of interest
    # target - pixel coordinates for which we want to find the closest white pixel in the region
    #
    # output arguments:
    # nearest_coords - coordinates of the nearest white pixel
    
    distances = np.sqrt((coords[:,0] - target[0]) ** 2 + (coords[:,1] - target[1]) ** 2)
    nearest_index = np.argmin(distances)
    
    nearest_coords = coords[nearest_index]
    
    return nearest_coords

In [ ]:
def jaccard_index (img_segmented,img_groundtruth):
    
    # input arguments:
    # img_segmented - segmented binary image
    # img_groundtruth - binary groundtruth image
    #
    # output arguments:
    # index - jaccard index 
    
    # find the number of true positives, false positives and false negatives, required for jaccard index computation
    TP = 0
    FP = 0
    FN = 0
    
    for i in range (img_segmented.shape[0]):
        for j in range (img_segmented.shape[1]):
            if (int(img_segmented[i][j]) > 0) & (int(img_groundtruth[i][j]) == 1):
                TP += 1
            elif (int(img_segmented[i][j]) > 0) & (int(img_groundtruth[i][j]) == 0):
                FP += 1
            elif (int(img_segmented[i][j]) == 0) & (int(img_groundtruth[i][j]) == 1):
                FN += 1
                
    index = TP / (TP + FP + FN)
    
    return index

In [ ]:
# load all original images
original_images_files = sorted(glob.glob(".\\dataset\\images\\*.jpg"))
original_images = [cv2.imread(file) for file in original_images_files]

# load all groundtruth images
groundtruth_files = sorted(glob.glob(".\\dataset\\groundtruths\\*.png"))
groundtruths = [cv2.imread(file) for file in groundtruth_files]
for i in range (len(groundtruths)):
    groundtruths[i] = color.rgb2gray(groundtruths[i])

In [ ]:
# define kernel for dilation
kernel = np.ones((20,20),np.float32)/400

result_images_meanshift = original_images.copy() 

In [ ]:
# create a new folder to save the results
path_meanshift = '.\\meanshift\\' 
if not os.path.exists(path_meanshift):
    os.makedirs(path_meanshift)

In [ ]:
# meanshift segmentation

for i in range (len(original_images)):
    result_images_meanshift[i] = hair_removal(original_images[i])
    result_images_meanshift[i] = meanshift_segmentation(result_images_meanshift[i])   
    result_images_meanshift[i] = region_extraction(result_images_meanshift[i])
    result_images_meanshift[i] = cv2.dilate(result_images_meanshift[i],kernel,iterations = 1) 
    cv2.imwrite(path_meanshift + os.path.basename(original_images_files[i]), result_images_meanshift[i])


In [ ]:
# compute the jaccard index for the meanshift segmented images
index_meanshift = []
for i in range (len(result_images_meanshift)):
    index_meanshift.append(jaccard_index(result_images_meanshift[i],groundtruths[i]))
    
plt.plot(index_meanshift)
plt.show()
print(index_meanshift)
print(np.mean(index_meanshift))    

In [ ]:
# load all original images
original_images_files = sorted(glob.glob(".\\dataset\\images\\*.jpg"))
original_images = [cv2.imread(file) for file in original_images_files]

# load all groundtruth images
groundtruth_files = sorted(glob.glob(".\\dataset\\groundtruths\\*.png"))
groundtruths = [cv2.imread(file) for file in groundtruth_files]
for i in range (len(groundtruths)):
    groundtruths[i] = color.rgb2gray(groundtruths[i])

In [ ]:
# define kernel for dilation
kernel = np.ones((20,20),np.float32)/400

result_images_watershed = original_images.copy()

In [ ]:
# create a new folder to save the results
path_watershed = '.\\watershed\\' 
if not os.path.exists(path_watershed):
    os.makedirs(path_watershed)

In [ ]:
# watershed segmentation
for i in range (len(original_images)):
    result_images_watershed[i] = hair_removal(original_images[i])
    result_images_watershed[i] = watershed_segmentation(original_images[i],result_images_watershed[i])   
    result_images_watershed[i] = region_extraction(result_images_watershed[i])
    result_images_watershed[i] = cv2.dilate(result_images_watershed[i],kernel,iterations = 1)    
    cv2.imwrite(path_watershed + os.path.basename(original_images_files[i]), result_images_watershed[i])       

In [ ]:
# compute the jaccard index for the watershed segmented images
index_watershed = []
for i in range (len(result_images_watershed)):
    index_watershed.append(jaccard_index(result_images_watershed[i]/255,groundtruths[i]))
    
plt.plot(index_watershed)
plt.show()
print(index_watershed)
print(np.mean(index_watershed)) 